Part #1 - Some Global Functions

Function A - Importing Market Data

In [57]:
# n = Name of the excel file
# s = Sheet Name within the excel file
# r = Range of data which needs to be imported
# d = Data object that needs to be sent while exporting back

def import_excel(n,s,r):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    data = sheet.range(r).value
    return data 

def export_excel(n,s,r,d):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    sheet.range(r).value = d




In [ ]:
data = import_excel('Market Data.xlsx','USDIRS','A3:B12')
#print(data)

[[1.0, 0.0407498], [2.0, 0.037767], [3.0, 0.037051], [4.0, 0.03707], [5.0, 0.0373805], [6.0, 0.037867], [7.0, 0.0383775], [8.0, 0.0388615], [9.0, 0.0393125], [10.0, 0.0397465]]


Function B - MFBD Holiday Calendar

In [4]:
#dt = Date on which operation is to be performed
#cal1 = Country of Calendar 1, For USA pass argument US
#cal2 = Country of Calendar 2, For India pass argument IN .. This is an optional argument
# At present this function suports a max of 2 calendars only for MFBD

def mfbd(dt,cal1, cal2="NIL"):
    from datetime import date
    from datetime import timedelta
    import holidays
    calendar1 = holidays.country_holidays(cal1)
    if cal2 == "NIL":
        while dt.weekday() > 4 or dt in calendar1:
            dt = dt + timedelta(days=1)
    else:
        calendar2 = holidays.country_holidays(cal2)
        while dt.weekday() > 4 or dt in calendar1 or dt in calendar2:
            dt = dt + timedelta(days=1)
    #dt = date(2025,5,24)
    return dt


Part #2 - Compute the Maturity Date and Payment Date for Swaps

In [16]:
from datetime import date,timedelta
import holidays
import pandas as pd

#PricingDate = date.today() #Use todays date as the Pricing Date
PricingDate = date(2025,5,19) # For testing currently
SDate = mfbd(PricingDate + timedelta(days=2),'US') #Calculating the Spot Start Date
#print(PricingDate)
#print(SDate)
#print(MatYears)
#print(SDate.year)
df = pd.DataFrame(data)
df.columns = ['Year','Rate']
#df.interpolate()
#print(df)
df['MatDate'] = SDate 
df['PmtDate'] = SDate

#Populate Maturity Date and Payment Date adjusting for pay delay

for index,row in df.iterrows():
    df.at[index,'MatDate'] = mfbd(row['MatDate'] + pd.offsets.DateOffset(years=row['Year']),'US')
    df.at[index,'PmtDate'] = mfbd(df.at[index,'MatDate'] + pd.offsets.DateOffset(days=2),'US') #SOFR Swaps have 2 days Payment Delay
    
   
df['MatDate'] = pd.to_datetime(df['MatDate']).dt.date # To remove hh:mm:ss from Date
df['PmtDate'] = pd.to_datetime(df['PmtDate']).dt.date
df['DC'] = df['PmtDate'].diff(periods=1) #Compute Periodic day counts
df.at[0,'DC']= pd.to_timedelta(df.at[0,'PmtDate']-SDate) #Compute first DC value from Start Date
df['DC']=pd.to_timedelta(df['DC']).dt.days.astype(float)/360 #Since Daycount is Act/360 for USD SOFR
#df['DC'] = df['DC']
#for each i in df['Year']:
#    df['MatDate'] = SDate + pd.offsets.DateOffset(years=i)

#df['MatDate'] = date(SDate.year + df['Year'],SDate.month,SDate.day)
#print(df)    
#df.interpolate()



Part #3 - Compute Discount Factors and Zeros (Aka Curve Bootstrapping)

In [28]:
import numpy as np
df['DF'] = 1
df['DF'] = df['DF'].astype(float)
SDate = mfbd(SDate,'US')

#Just testing, delete later
for index,row in df.iterrows():
    df.at[index,'DF'] = 1/(pow(1+row['Rate'],row['Year']))

#print(df)
#dd = (df.at[0,'MatDate'] - SDate).days
#print(dd)
#df.at[0,'DF'] = 1/(1+df.at[0,'Rate']*(df.at[0,'PmtDate']-SDate).days/360)

#Compute actual Curve DFs for USD SOFR
#The for loop is massively simplified to solve a linear system of equations. Will write a full document on how this is calculated later. For queries in the interim email pushkargondane@gmail.com 
dfdccumprodsum = 0
for index,row in df.iterrows():
    df.at[index,'DF'] = (1 - df.at[index,'Rate']*dfdccumprodsum)/(1+df.at[index,'Rate']*df.at[index,'DC'])
    dfdccumprodsum = dfdccumprodsum + df.at[index,'DC']*df.at[index,'DF']

#Compute Zeros for USD SOFR
# Please note that zeros requires an assumption of the approach. Here we are going to try and match
# it with that of bloomberg by assuming continuous compounding
# Different systems implement this bit in a different manner


df['Zero']=1
df['Zero'] = df['Zero'].astype(float)
for index,row in df.iterrows():
    p = pd.to_timedelta(df.at[index,'PmtDate']-df.at[index,'MatDate'])
    t = float(p.days)/365 + df.at[index,'Year']
    #print(t)
    df.at[index,'Zero'] = np.log(1/df.at[index,'DF'])/t
    


print(df)


   Year      Rate     MatDate     PmtDate        DC        DF      Zero
0   1.0  0.040750  2026-05-21  2026-05-26  1.027778  0.959802  0.040474
1   2.0  0.037767  2027-05-21  2027-05-24  1.008333  0.927426  0.037517
2   3.0  0.037051  2028-05-22  2028-05-24  1.016667  0.895086  0.036878
3   4.0  0.037070  2029-05-21  2029-05-23  1.011111  0.862696  0.036873
4   5.0  0.037380  2030-05-21  2030-05-23  1.013889  0.830086  0.037204
5   6.0  0.037867  2031-05-21  2031-05-23  1.013889  0.797265  0.037727
6   7.0  0.038378  2032-05-21  2032-05-24  1.019444  0.764618  0.038295
7   8.0  0.038862  2033-05-23  2033-05-25  1.016667  0.732701  0.038851
8   9.0  0.039313  2034-05-22  2034-05-24  1.011111  0.701706  0.039336
9  10.0  0.039746  2035-05-21  2035-05-23  1.011111  0.671430  0.039813


Part #4 - Validate DF vs Alternate System like Bloomberg

![title][def]

[def]: SOFRBBG.png

In [70]:
bbgdata = import_excel('Market Data.xlsx','USDIRS','g3:h12')
bbg = pd.DataFrame(bbgdata)
bbg.columns=['BBGDF','BBGZero']
#print(bbg)
comp = df[['Year','MatDate','PmtDate','DF','Zero']]
#comp.append(bbg)
#comp.join(bbg)
comp['BBGDF'] = bbg['BBGDF'] #Need to find a better method to append columns as this is giving a warning 
#comp.assign(bbgdata)
#comp['BBGDF'] = bbg.loc[:,'BBGDF']
comp['BBGZero'] = bbg['BBGZero']
#comp['BBGZero'] = bbg.loc[:,'BBGZero']
comp['DF Gap(bps)']=(comp['BBGDF']-comp['DF'])*10000/comp['BBGDF']
comp['Zero Gap(bps)']=(comp['BBGZero']-comp['Zero'])*10000/comp['BBGZero']
print(comp)
comp=comp.drop(columns=['Year','BBGDF','BBGZero','DF Gap(bps)','Zero Gap(bps)'])

   Year     MatDate     PmtDate        DF      Zero     BBGDF   BBGZero  \
0   1.0  2026-05-21  2026-05-26  0.959802  0.040474  0.960094  0.040502   
1   2.0  2027-05-21  2027-05-24  0.927426  0.037517  0.927482  0.037538   
2   3.0  2028-05-22  2028-05-24  0.895086  0.036878  0.895045  0.036826   
3   4.0  2029-05-21  2029-05-23  0.862696  0.036873  0.862658  0.036859   
4   5.0  2030-05-21  2030-05-23  0.830086  0.037204  0.830051  0.037193   
5   6.0  2031-05-21  2031-05-23  0.797265  0.037727  0.797234  0.037716   
6   7.0  2032-05-21  2032-05-24  0.764618  0.038295  0.764668  0.038271   
7   8.0  2033-05-23  2033-05-25  0.732701  0.038851  0.732677  0.038802   
8   9.0  2034-05-22  2034-05-24  0.701706  0.039336  0.701686  0.039303   
9  10.0  2035-05-21  2035-05-23  0.671430  0.039813  0.671413  0.039794   

   DF Gap(bps)  Zero Gap(bps)  
0     3.043136       6.936892  
1     0.600978       5.616469  
2    -0.453521     -14.162227  
3    -0.444027      -3.806096  
4    -0.424608

/var/folders/5g/4br4skc15dlb750j33xn06rh0000gn/T/ipykernel_1869/3610128809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['BBGDF'] = bbg['BBGDF'] #Need to find a better method to append columns as this is giving a warning


Note that while the gap in DFs is miniscule (less than 1 bps vs BBG DFs) the zeros are slightly more different. They dont impact any calculations as the approach to calculate zeros simply depends on the base formula used for zeros. 

Valuation differences may arise if you choose to interpolate on zeros vs interpolate on DFs 

Part #5 - Sending the DFs and Zeros calculated back to excel

In [71]:
export_excel('Market Data.xlsx','USDIRS','L2:L11',comp)

Part #6 - Pricing an Interest Rate Swap

